In [394]:
from bs4 import BeautifulSoup
import requests
import time
from urllib.request import urlopen, Request
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)

/var/folders/c7/cnkrdqk12hj7hh7h82qw9xg80000gn/T/ipykernel_2475/1805676031.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)
/var/folders/c7/cnkrdqk12hj7hh7h82qw9xg80000gn/T/ipykernel_2475/1805676031.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)


In [ ]:
import urllib.request

headers = {'User-Agent': 'Mozilla/5.0'}

book_titles = []
book_descriptions = []
book_genres = []
book_authors = []

place = 0

page_link = "https://www.goodreads.com/list/show/9.Best_Books_of_the_Decade_1980s?page=1"
driver.get(page_link)
soup = BeautifulSoup(driver.page_source, 'html.parser')

for i in range(50):
    books = soup.find_all('a', class_='bookTitle')
    for i in books:
        link = "https://goodreads.com" + i['href']
        time.sleep(2)
        resp = requests.get(link, headers=headers)

        book_soup = BeautifulSoup(resp.content, 'html.parser')
        description = book_soup.find(class_="DetailsLayoutRightParagraph__widthConstrained")
        if description == None:
            if book_soup.find(class_="readable stacked") == None:
                description = "No description"
            else:
                if book_soup.find(class_="readable stacked").find('span', style="display:none") == None:
                    description = "No description"
                    break 
                description = book_soup.find(class_="readable stacked").find('span', style="display:none").get_text()     
        else:
            description = description.text     
        book_descriptions.append(description)
        f2 = open("book_descriptions2.txt", "a")
        f2.write(str(place) + " " + description + "\n" + "\n")
        f2.close() 
        book_title = book_soup.find("meta", property="og:title")
        if book_title == None:
            book_title = "No title"
        else:
            book_title = book_title.get('content')  
        book_titles.append(book_title)
        f = open("book_titles2.txt", "a")
        f.write(str(place) + " " + book_title + "\n")
        f.close() 
        genre_buttons = []
        genre_buttons = book_soup.find_all(class_="actionLinkLite bookPageGenreLink")
        if(len(genre_buttons) == 0):
            genre_buttons = book_soup.find_all(class_='BookPageMetadataSection__genreButton')
        single_book_genres = []
        genre_i = 0
        f3 = open("book_genres2.txt", "a")
        f3.write(str(place) + " ")
        for j in genre_buttons:
            if(genre_i == 5):
                break
            single_book_genres.append(j.text)
            genre_i += 1
            f3.write(j.text + ", ")
        book_genres.append(single_book_genres) 
        f3.write("\n")
        f3.close()
        author_name = book_soup.find(class_="authorName")
        if author_name == None:
            if book_soup.find(class_="ContributorLink__name") == None:
                author_name = "No author"
                break
            author_name = book_soup.find(class_="ContributorLink__name").get_text()
        else:
            author_name = author_name.get_text()
        book_authors.append(author_name)
        f4 = open("book_authors2.txt", "a")
        f4.write(str(place) + " " + author_name + "\n")
        f4.close()
        place += 1
               
    page_link = 'https://goodreads.com' + soup.find('a', class_='next_page').get('href')
    driver.get(page_link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    


In [47]:
string_add = ""
book_titles = []
titles = open("book_titles2.txt", "r")

#book_titles
while True:
    line = titles.readline()
  
    if line == 'END':
        break
    if line == '\n':
        continue
    
    if line.split()[0].isdigit():
        if string_add.strip() != "":
            book_titles.append(string_add)
        string_add = line.lstrip('0123456789').strip()
    else:
        string_add += line.strip()

if string_add != "":
    book_titles.append(string_add)     
titles.close() 
print(len(book_titles))

5905


In [83]:
string_add = ""
authors = open("book_authors2.txt", "r")
book_authors = []

#book_authors
while True:
    line = authors.readline()
  
    if line == 'END':
        break
    if line == '\n':
        continue
    
    if line.split()[0].isdigit():
        if string_add.strip()  != "":
            book_authors.append(string_add)
        string_add = line.lstrip('0123456789').strip()
    else:
        string_add += line.strip()

if string_add != "":
    book_authors.append(string_add)     
authors.close() 
print(len(book_authors))

5905


In [65]:
string_add = ""
descriptions = open("book_descriptions2.txt", "r")
book_descriptions = []
idx = 0
#book_descriptions
while True:
    line = descriptions.readline()
    if line == '' or line == "END":
        break
    if line == '\n':
        continue
    
    if line.split()[0].isdigit():
        if string_add != "":
            book_descriptions.append(string_add)
        string_add = line.lstrip('0123456789').strip()
    else:
        string_add += line.strip()

if string_add != "":
    book_descriptions.append(string_add)     
descriptions.close() 
print(len(book_descriptions))   


5905


In [81]:
string_add = ""
genres = open("book_genres2.txt", "r")
book_genres = []

#book_genres
while True:
    line = genres.readline()
    if line == "END":
        break
    if line == '\n':
        continue
    
    if line.split()[0].isdigit():
        string_add = line.lstrip('0123456789').strip().rstrip(',')
        if string_add == "":
            string_add = "No genre"
        book_genres.append(string_add.split(', '))
        
genres.close() 
print(len(book_genres))


5905


In [105]:
#get rid of no genre and no description

indexes = []

for i in range(len(book_genres)):
    if book_genres[i][0] == 'No genre':
        indexes.append(i)
for i in range(len(book_descriptions)):
    if book_descriptions[i] == "No description":
        indexes.append(i)
res = [*set(indexes)]

book_genres_cleaned = []
book_descriptions_cleaned = []
book_authors_cleaned = []
book_titles_cleaned = []

for i in range(len(book_genres)):
    if i not in res:
        book_genres_cleaned.append(book_genres[i])
        book_descriptions_cleaned.append(book_descriptions[i])
        book_authors_cleaned.append(book_authors[i])
        book_titles_cleaned.append(book_titles[i])

print(len(book_genres_cleaned))
print(len(book_descriptions_cleaned))
print(len(book_authors_cleaned))
print(len(book_titles_cleaned))

5605
5605
5605
5605
